<a href="https://colab.research.google.com/github/jjomathew98/sqlite_database_operations/blob/main/sqlite_database_operations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Data Exploration and Analysis
### Loading Packages - Pandas

In [43]:
import pandas as pd

# 1.1 Import Pricing Transparency Datasets

In [44]:
df1 = pd.read_json('https://raw.githubusercontent.com/hantswilliams/HHA_504_2023/main/WK3/data/nyp/133957095_NewYorkPresbyterianHospital_standardcharges.json')
df2 = pd.read_csv('https://raw.githubusercontent.com/hantswilliams/HHA_504_2023/main/WK3/data/stonybrook/stonybrook.csv')

# 1.2 Conduct a Basic Exploratory Analysis
## Dataset #1

In [45]:
df1.head()
df1.info()
df1.describe()
df1['1199'].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6193 entries, 0 to 6192
Data columns (total 47 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Code (CPT/DRG)                      6193 non-null   object 
 1   Description                         6193 non-null   object 
 2   Rev Code                            6193 non-null   object 
 3   Inpatient/Outpatient                6193 non-null   object 
 4   Gross Charges                       6193 non-null   float64
 5   Discounted Cash Price               6193 non-null   float64
 6   Aetna                               6193 non-null   object 
 7   Cigna                               6193 non-null   object 
 8   Empire Blue Cross Blue Shield       6193 non-null   object 
 9   Emblem Health                       6193 non-null   object 
 10  United Health Group                 6193 non-null   object 
 11  Aetna Medicare                      6193 no

2107                           375
2412                           343
1564                           236
Included in Service Package    235
2965                           189
                              ... 
601                              1
202.5                            1
197                              1
389                              1
17670.3696                       1
Name: 1199, Length: 2230, dtype: int64

# 1.2 Conduct a Basic Exploratory Analysis
## Dataset #2

In [46]:
df2.head()
df2.info()
df2.describe()
df2['1199-Commercial other'].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7710 entries, 0 to 7709
Data columns (total 57 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   Code                                          7710 non-null   object 
 1   Description                                   7710 non-null   object 
 2   Type                                          6927 non-null   object 
 3   Package/Line_Level                            6927 non-null   object 
 4   Gross charge                                  7710 non-null   float64
 5   Discounted cash price                         7710 non-null   float64
 6   De-identified min contracted rate             6254 non-null   float64
 7   De-identified max contracted rate             6254 non-null   float64
 8   Derived contracted rate                       6254 non-null   float64
 9   1199-Commercial other                         1014 non-null   f

4822.00     104
2411.00      36
2153.00      36
4592.00      27
3923.00      19
           ... 
221.63        1
14.40         1
185.40        1
157.20        1
42565.43      1
Name: 1199-Commercial other, Length: 568, dtype: int64

# 2. SQLite Database Operations

In [47]:
import sqlite3

## 2.1 Create SQLite Database

In [48]:
conn = sqlite3.connect('health.db')

## 2.2 Manual Table Creation and Data Insertion

In [49]:
cursor = conn.cursor()

In [50]:
# Create a table
create_table_query = '''
CREATE TABLE IF NOT EXISTS hospital_data (
    id INTEGER PRIMARY KEY,
    numerical_col1 REAL,
    numerical_col2 REAL,
    categorical_col1 TEXT,
    categorical_col2 TEXT
);
'''
cursor.execute(create_table_query)

# Insert data into the table
insert_data_query = '''
INSERT INTO hospital_data (numerical_col1, numerical_col2, categorical_col1, categorical_col2)
VALUES (?, ?, ?, ?);
'''
cursor.execute(insert_data_query, (1.0, 2.0, 'Category1', 'Category2'))
cursor.execute(insert_data_query, (3.0, 4.0, 'Category3', 'Category4'))

# Commit changes and close the connection
conn.commit()
conn.close()


# 2.3 Automatic Table Creation (df1)


In [52]:
conn = sqlite3.connect('health.db')
df1.to_sql('hospital_data_auto', conn, if_exists='replace', index=False)
conn.close()

# Dive Deeper with SQL (Custom Select Query)

In [53]:
conn = sqlite3.connect('health.db')
query = '''
SELECT categorical_col1, AVG(numerical_col1) AS avg_num_col1
FROM hospital_data
GROUP BY categorical_col1;
'''
result = pd.read_sql_query(query, conn)
conn.close()

# Print the result or perform further analysis
print(result)

  categorical_col1  avg_num_col1
0        Category1           1.0
1        Category3           3.0
